In [15]:
import cv2
import os
import pandas as pd
import numpy as np

def fit_ellipse(im):    
    imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imgray,127,255,0)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    contour_sizes = [len(contour) for contour in contours]
    ellipse = cv2.fitEllipse(contours[np.argmax(contour_sizes)])
    return ellipse

In [40]:
df_pixel_size = pd.read_csv('data/HC18/test_set_pixel_size.csv', index_col='filename')
print(df_pixel_size.head())
submission = {'filename': [], 'center_x_mm': [], 'center_y_mm': [], 'semi_axes_a_mm': [],
             'semi_axes_b_mm': [], 'angle_rad': [], 'Index': []}

file_names = sorted([file for _,_,files in os.walk('data/HC18/test_set/results/') for file in files])

for i, file_name in enumerate(file_names):
    im = cv2.imread('data/HC18/test_set/results/'+file_name)
   # file_name = file_name.replace('_Segmented', '')
    ellipse = fit_ellipse(im)
    im_mask = cv2.ellipse(np.zeros_like(im),ellipse,(255,255,255),-1)
    image_name = file_name.split('.')[0] + '_Annotation_fit.png'
    cv2.imwrite('data/HC18/test_set/results_masked/'+image_name, im_mask)
    file_name = file_name.replace('_predict', '')

    pixel_size = df_pixel_size.loc[file_name, 'pixel size(mm)']
    submission['filename'].extend([file_name])
    submission['Index'].extend([int(file_name.split('_')[0])])
    center_x,  center_y = ellipse[0]
    submission['center_x_mm'].extend([pixel_size*center_x])
    submission['center_y_mm'].extend([pixel_size*center_y])
    semi_axes_b, semi_axes_a = ellipse[1]
    if semi_axes_b > semi_axes_a:
        semi_axes_b = semi_axes_b + semi_axes_a
        semi_axes_a = semi_axes_b - semi_axes_a
        semi_axes_b -= semi_axes_a
    submission['semi_axes_a_mm'].extend([semi_axes_a*pixel_size/2])
    submission['semi_axes_b_mm'].extend([semi_axes_b*pixel_size/2])
    angle = ellipse[2]
    if angle < 90:
        angle += 90
    else:
        angle -= 90
    submission['angle_rad'].extend([np.deg2rad(angle)])

submission = pd.DataFrame(submission)
submission = submission[['filename', 'center_x_mm', 'center_y_mm', 
                         'semi_axes_a_mm', 'semi_axes_b_mm', 'angle_rad', 'Index']]    
submission['center_x_mm'] = submission['center_x_mm'].tolist()
print(submission.head())
submission = submission.sort_values(['Index'])
submission = submission.drop('Index', axis=1)
submission.to_csv("midterm.csv", index=False)

            pixel size(mm)
filename                  
000_HC.png        0.235857
001_HC.png        0.067973
002_HC.png        0.165202
003_HC.png        0.096674
004_HC.png        0.202203
     filename  center_x_mm  center_y_mm  semi_axes_a_mm  semi_axes_b_mm  \
0  000_HC.png    93.253996    64.819197       55.147874       41.128129   
1  001_HC.png    29.987074    21.423988       12.275935       10.433662   
2  002_HC.png    60.614400    42.016332       38.888171       29.039810   
3  003_HC.png    34.815495    27.227664       31.276386       22.759055   
4  004_HC.png    81.873756    57.953754       60.744811       43.196498   

   angle_rad  Index  
0   3.137891      0  
1   3.130502      1  
2   0.044414      2  
3   3.086192      3  
4   0.048679      4  
